In [ ]:
!pip install torch torch_geometric awkward gdown matplotlib numpy scipy networkx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/A3D3_Workshop_2023_Dadarlatlab/A3D3_Hackathon_202307_Preprocessed

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1kFaLNqckbXtQ8RgJwPDjKW_PoX5e_Kyi/A3D3_Workshop_2023_Dadarlatlab/A3D3_Hackathon_202307_Preprocessed


In [ ]:
!gdown --folder --id 1OUyLYTRGpM9KZWq6g1hFzcWu76EOldcz
!gdown --folder --id 1mI0cTkS3BT2AlySiWaw51uUYlnMNwokd

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder list
Retrieving folder 1f7w_ZpYoj5kRMWD67X1GN_vxu-5M-_9e Behavior
Processing file 15F5TY8QXaqB_zltGGpLlvchiOdHIQF90 limb1.avi
Processing file 1bO1utwDxoyUoGHX2yjj385Uhz5bCR8Az limb2.avi
Processing file 14gI2nNTv2_jG9kPgBce8a4vOnaWF6rpW touch_timestamps.csv
Processing file 1vA1dljNg_AjUQkAPEyh2T-xCS6z86bP2 ffneu_final_neural_avg.png
Processing file 1vA9XyGLnKMqx5s3ITk5_6kVijdU7ivX_ ffneu_final_neural.png
Processing file 1ubHL4a_kuSTU2sMcz4FVg2iKXawSbmOz ffneu_final.npy
Processing file 1ucpN2Rco5yqR40N2qVO1UbhHJH931p-I idx_coord_neural.npy
Processing file 1uxDj9PG3kU5x8kA_AjpR6X31B14RpC-0 number_frames_hist.png
Processing file 1umRjyumSvoFUIeXMxXdo0Cx0GN_awZB0 spks_final_neural_avg.png
Processing file 1uwOyZ9WMXOZeJYZseCJ9Avi1jmFVDLCY spks_final_neural.png
P

In [ ]:
import os
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import networkx as nx

from scipy.spatial.distance import cdist
import numpy as np
import awkward as ak

In [ ]:
def load_npy_files(directory):
    data_dict = {}  # Initialize an empty dictionary to store the data
    for file_name in os.listdir(directory):  # Iterate over all files in the directory
        if file_name.endswith('.npy'):  # Check if the file has a .npy extension
            file_path = os.path.join(directory, file_name)  # Get the full path of the file
            data = np.load(file_path, allow_pickle=True)  # Load the numpy array from the file
            key = os.path.splitext(file_name)[0]  # Extract the file name without extension as the dictionary key
            data_dict[key] = data  # Add the data to the dictionary with the corresponding key
    return data_dict

# Usage example
directory_path = 'Animal1_Touch'  # Replace with the actual directory path
train_set = load_npy_files(directory_path)  # Load .npy files from the specified directory
train_set['stat'] = ak.from_iter(train_set['stat']) #convert from regular dictionary to awkward array


directory_path = 'Animal2_Touch'  # Replace with the actual directory path
test_set = load_npy_files(directory_path)  # Load .npy files from the specified directory
test_set['stat'] = ak.from_iter(test_set['stat'])

print(test_set.keys())

dict_keys(['touch_behav', 'ffneu_final', 'idx_coord_neural', 'stat', 'spks_final'])


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_set, time_threshold, distance_threshold, transform=None, pre_transform=None):
        super(MyDataset, self).__init__(".", transform, pre_transform)
        self.data_set = data_set
        self.time_threshold = time_threshold
        self.distance_threshold = distance_threshold

        stat_med = torch.tensor(data_set['stat']['med'])
        self.x_coord, self.y_coord = stat_med.T
        self.prelabel = torch.tensor(data_set['touch_behav'][:, 2])


    def create_edges_within_distance(self, points, distance_threshold):
        x, y = points[0], points[1]
        euclidean_distances = torch.sqrt(torch.pow(x[:, None] - x[None, :], 2) +
                                         torch.pow(y[:, None] - y[None, :], 2)).to(torch.float32)

        indices = torch.where(euclidean_distances <= distance_threshold)
        edges = torch.stack(indices, dim=0)
        edge_distances = euclidean_distances[indices]
        return edges, edge_distances

    @property
    def raw_file_names(self):
        return []  # Not used in this example

    @property
    def processed_file_names(self):
        return []  # Not used in this example

    def len(self):
        return len(self.data_set['idx_coord_neural'])

    def get(self, i):
        idx = self.data_set['idx_coord_neural'][i]

        spks_final = self.data_set['spks_final'][:, idx]
        mask = spks_final > self.time_threshold
        x_coord = self.x_coord[mask].to(torch.float32)
        y_coord = self.y_coord[mask].to(torch.float32)
        t_spike = torch.tensor(spks_final[mask]).to(torch.float32)

        x = torch.stack((x_coord, y_coord, t_spike), dim=1)  # Create the node features tensor

        edge_index, edge_attr = self.create_edges_within_distance(x, self.distance_threshold)

        label = self.prelabel[(self.data_set['touch_behav'][:, 0] <= i * 4) & (i * 4 <= self.data_set['touch_behav'][:, 1])].tolist()
        label = label[0] if len(label) > 0 else 0

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, graph_attr=i)
        data.y = torch.tensor(label)

        return data

time_threshold = -1
distance_threshold = 10000

train_dataset = MyDataset(train_set, time_threshold, distance_threshold)
test_dataset = MyDataset(test_set, time_threshold, distance_threshold)

#to fully pregen dataset before training for speed up
# Can skip this if dataset is ram intensive
# train_dataset = [data for data in train_dataset]
# test_dataset = [data for data in test_dataset]

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=2)  # Adjust num_workers as needed
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=2)  # Adjust num_workers as needed

In [ ]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(3, 16)
        self.conv2 = GCNConv(16, 32)
        self.conv3 = GCNConv(32, 64)
        self.fc = torch.nn.Linear(64, 5)

    def forward(self, data):
        x, edge_index = data.x.double(), data.edge_index

        # Node-level feature transformations
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv3(x, edge_index))

        # Global-level feature extraction via mean pooling
        x = torch_geometric.nn.global_mean_pool(x, data.batch)

        # Output layer
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [ ]:
import torch
from torch_geometric.data import DataLoader
from torch.optim import Adam

# We're using a GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model and optimizer
model = Net().double().to(device)
optimizer = Adam(model.parameters(), lr=0.01)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training function
def train():
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output.to(device), torch.tensor(data.y).to(device))
        loss.backward()
        optimizer.step()
        loss_all += data.num_graphs * loss.item()
    return loss_all / len(train_dataset)

# Testing function
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data).max(dim=1)[1]
        correct += pred.eq(torch.tensor(data.y).to(device)).sum().item()
    return correct / len(loader.dataset)

# Training loop
for epoch in range(1, 101):
    train_loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch}, Train Loss: {train_loss}, '
          f'Train Acc: {train_acc}, Test Acc: {test_acc}')


<ipython-input-13-bcd8a8a4ab86>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.nll_loss(output.to(device), torch.tensor(data.y).to(device))
<ipython-input-13-bcd8a8a4ab86>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  correct += pred.eq(torch.tensor(data.y).to(device)).sum().item()


Epoch: 1, Train Loss: 0.16457785599956615, Train Acc: 0.9846804785526699, Test Acc: 0.9839499096080074


In [ ]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'

In [ ]:
for data in train_dataset:
    if data.y not in [0,1,2,3,4]:
        print(data)

In [ ]:
torch.tensor([data.y for data in train_dataset])